In [ ]:
# Incorporate Factual Data Using SERP API:
#I ntegrate real-time search results using the fetchCompetitors workflow. This helps ground the AI responses in factual information.

In [ ]:
!pip3 install google.analytics.data
!pip3 install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
    Filter,
    FilterExpression,
)
import os
import pandas as pd
import json
import scipy.stats as st
import numpy as np

In [ ]:
def sample_run_report(property_id="481920227"):
    """Runs a simple report on a Google Analytics 4 property."""
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = # json file
    client = BetaAnalyticsDataClient()
    request = RunReportRequest(
        property="properties/{}".format(property_id),
        dimensions=[Dimension(name="source"),Dimension(name="medium"),Dimension(name="eventName"),Dimension(name="date")], #Dimension(name="sessionCampaignName"),
        metrics=[Metric(name="sessions")],
        date_ranges=[DateRange(start_date="2025-03-20", end_date="2025-04-16")],
    )
    response = client.run_report(request)
    return response

In [ ]:
def response_to_df(response):
    columns = []
    rows = []

    for col in response.dimension_headers:
        columns.append(col.name)
    for col in response.metric_headers:
        columns.append(col.name)

    for row_data in response.rows:
        row = []
        for val in row_data.dimension_values:
            row.append(val.value)
        for val in row_data.metric_values:
            row.append(val.value)
        rows.append(row)
    return pd.DataFrame(rows, columns=columns)


response=sample_run_report()
df=response_to_df(response)

df = df[(df['eventName'] == 'submit') | (df['eventName'] == 'again')]
print(df)

              source      medium eventName      date sessions
0               text        text    submit  20250323       25
1           (direct)      (none)    submit  20250326       18
3         discussion      canvas    submit  20250416        5
4           (direct)      (none)     again  20250326        4
5           (direct)      (none)    submit  20250325        4
6         discussion      canvas    submit  20250415        4
7           (direct)      (none)    submit  20250323        2
8         discussion      canvas    submit  20250325        2
9         discussion      canvas    submit  20250326        2
11     tessbalentine     website    submit  20250401        2
12          (direct)      (none)     again  20250325        1
13          (direct)      (none)     again  20250414        1
15          (direct)      (none)    submit  20250414        1
16          (direct)      (none)    submit  20250416        1
18            Canvas    campaign    submit  20250325        1
21      

In [ ]:
# source
discussion = df[df['source'] == 'discussion']['sessions'].astype(int)
direct = df[df['source'] == '(direct)']['sessions'].astype(int)
text = df[df['source'] == 'text']['sessions'].astype(int)

In [ ]:
from pingouin import ttest

ttest(discussion,direct,correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.24858,10,two-sided,0.808716,"[-7.47, 5.97]",0.152223,0.493,0.055855


In [ ]:
ttest(discussion,direct,correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.343157,8.634424,two-sided,0.739687,"[-5.73, 4.23]",0.152223,0.502,0.055855


In [ ]:
ttest(discussion,text,correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.851618,5,two-sided,0.43331,"[-23.11, 11.61]",0.650434,0.667,0.107698


In [ ]:
ttest(discussion,text,correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.715612,2.035206,two-sided,0.547378,"[-39.76, 28.26]",0.650434,0.63,0.107698


In [ ]:
ttest(direct,text,correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.889825,9,two-sided,0.396721,"[-17.71, 7.71]",0.602414,0.643,0.125551


In [ ]:
ttest(direct,text,correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.605386,2.269254,two-sided,0.599967,"[-36.79, 26.79]",0.602414,0.57,0.125551


In [ ]:
# medium
canvas = df[df['medium'] == 'canvas']['sessions'].astype(int)
text = df[df['medium'] == 'text']['sessions'].astype(int)

In [ ]:
ttest(canvas,text,correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.347565,8,two-sided,0.737134,"[-13.36, 9.86]",0.224352,0.517,0.060934


In [ ]:
ttest(canvas,text,correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.430007,5.346728,two-sided,0.683975,"[-12.01, 8.51]",0.224352,0.527,0.060934


Both student t test and Welch’s t-test's p-value is higher than 0.05, which means that we should not reject the null hyphothesis that there is no difference between the submittion/again numbers each source/medium pair

In [ ]:
# Based on the feedback from others, identify an interface design issue (e.g., better output display).
# Design an AB testing strategy.

# Please implement your proposed AB test in a branch and specify the URL of the branch below. Please also discuss the following aspects of the AB test:

# How many subjects do you want to include?
# When do you want your AB test to stop?
# How do you measure the success of your AB test?
# What is your hypothesis?
# What would be the next AB test to perform, depending on the outcome of your AB test?

- Identify an interface design issue： "Search Again" is in the original page,
users can see this before searching anything. This can cause confusion.

New link: https://zhizuohw1_testbranch--721-group1.netlify.app/


- **Design an AB testing strategy:**

    Control Group (A): "Search Again" is visible on the initial page (current
version).

    Treatment Group (B): "Search Again" is only shown after search results are
returned, placed under the feedback section.

- **Baseline performance:**current engatment time in results page

- **Expected variant performance:** targeting an increase to 30%

- **Hypothesis:** Moving the "Search Again" button to appear only after results
will reduce user confusion and increase engagement time in the result page.


- Desired power: 0.8

- Significance level (alpha): 0.05

In [ ]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.power import TTestIndPower

mean_A = 50
mean_B = 50*1.3
std = 30

effect_size = (mean_B - mean_A) / std

analysis = TTestIndPower()
sample_size_per_group = analysis.solve_power(
    effect_size=effect_size,
    power=0.8,
    alpha=0.05,
    alternative='two-sided'
)

print(f"Required sample size per group: {int(sample_size_per_group)}")

Required sample size per group: 63


- How many subjects do you want to include? **63**

- When do you want your AB test to stop?

 **Suppose we have 15 users per day, we need run the test for 5 days**

- How do you measure the success of your AB test?
  If we reject the null hyphothesis and find there is a significant increase
  of eneagament time after moving the button under the results.

- What would be the next AB test to perform, depending on the outcome of your
AB test?

   Evaluate label clarity for feedback buttons.

   Control: Feedback text = “Was this result helpful?”

  Treatment: Feedback text = “Rate these results”


